## A. Configuraciones Generales.

In [ ]:
#1. Librerías.
%run "./librerias.ipynb"

In [ ]:
#2. Constantes.
#i. Generales.
%run "./constantes.ipynb"

#ii. Particulares.
mes_train = mes_train_ult_anio

dataset_input = datasets_path + 'competencia_02_fe_all_1.parquet' 
modelo = modelos_path + 'lgbm_all_1_undersampling.txt'
dataset_output = salidas_exp_colaborativos_path + 'modelo_1.parquet'

In [ ]:
#3. Funciones.
%run "./funciones.ipynb"

In [ ]:
#4. Lectura de dataframes post-feature engineering (.csv o .parquet), y modelos ya optimizados previamente (.txt).
#i. Modelo LGBM propio con 3 meses para calculo de slope y valor actual vs promedio histórico. 
data = pd.read_parquet(dataset_input)

In [ ]:
#5. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)
#ii. Elimino columnas de último momento por Data Concept.
columnas_de_interes_prestamos = data.filter(like='prestamos_personales').columns
data.drop(columnas_de_interes_prestamos,axis=1,inplace=True)

In [ ]:
#6. Pesos y reclusterización.
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data['clase_binaria2'] = 0
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#7. Dividimos entre conjuntos de datos.
#i. Datos para entrenar todo el modelo final para Kaggle.
train_data = data[data['foto_mes'].isin(mes_train)]

X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

In [ ]:
#8. Obtenemos los hiperparámetros del modelo.
model_lgb = lgb.Booster(model_file=modelo)
params = model_lgb.params # Parámetros.
best_iter = params["num_iterations"]  # Mejor número de iteraciones guardado en el modelo.

In [ ]:
#9. Dataframe con predicciones finales.
stacking_predictions = pd.DataFrame(columns=['numero_de_cliente', 'foto_mes', 'probabilidad'])

## B. Entrenamiento de Modelo Individual, y predicción de las folds restantes.

In [ ]:
#1. Entrenamos y predecimos.
#i. Se crean las 5 folds estratificadas por mes.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=semillass[0])

#ii. Se recorre cada fold, dividiendo entre 4 para train y 1 para validación.
for fold, (train_index, val_index) in enumerate(skf.split(X_train, train_data['foto_mes'])):
    print(f"Fold {fold + 1}")
    
    #a. Se dividen los datos en conjuntos de entrenamiento y validación para este fold.
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train_binaria2.iloc[train_index], y_train_binaria2.iloc[val_index]
    w_train_fold, w_val_fold = w_train.iloc[train_index], w_train.iloc[val_index]
    
    #b. Se crea el dataset de entrenamiento para LightGBM con el fold actual.
    train_data_fold = lgb.Dataset(X_train_fold, label=y_train_fold, weight=w_train_fold)
    
    #c. Entrenamiento del modelo en los datos del fold actual usando los parámetros del modelo original.
    model_lgb = lgb.train(params, train_data_fold, num_boost_round=best_iter)
    
    #d. Se predice el conjunto de validación para el fold actual.
    val_predictions = model_lgb.predict(X_val_fold)
    
    #e. Se crea un DataFrame temporal para almacenar las predicciones2.
    fold_predictions = pd.DataFrame({
        'numero_de_cliente': train_data.iloc[val_index]['numero_de_cliente'].values,
        'foto_mes': train_data.iloc[val_index]['foto_mes'].values,
        'probabilidad': val_predictions
    })
    
    #f. Se agregan las predicciones del fold al DataFrame final.
    stacking_predictions = pd.concat([stacking_predictions, fold_predictions], ignore_index=True)

In [ ]:
#2. Renombramos.
stacking_predictions.rename({"probabilidad":"probabilidad_modelo1"},axis=1,inplace=True)

In [ ]:
#3. Exportamos.
stacking_predictions.to_parquet(dataset_output)